In [31]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [32]:
file = tf.keras.utils.get_file('data.txt' , 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [44]:
text = open(file, 'rb').read().decode(encoding = 'utf-8').lower()

In [45]:
char = sorted(set(text))
char

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [49]:
char_to_ind = dict((c , i) for i, c in enumerate(char))
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'k': 23,
 'l': 24,
 'm': 25,
 'n': 26,
 'o': 27,
 'p': 28,
 'q': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'x': 36,
 'y': 37,
 'z': 38}

In [50]:
ind_to_char = dict((c , i) for c, i in enumerate(char))
ind_to_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'a',
 14: 'b',
 15: 'c',
 16: 'd',
 17: 'e',
 18: 'f',
 19: 'g',
 20: 'h',
 21: 'i',
 22: 'j',
 23: 'k',
 24: 'l',
 25: 'm',
 26: 'n',
 27: 'o',
 28: 'p',
 29: 'q',
 30: 'r',
 31: 's',
 32: 't',
 33: 'u',
 34: 'v',
 35: 'w',
 36: 'x',
 37: 'y',
 38: 'z'}

In [37]:
sent_length = 100
step_size = 3
sent = []
new_char =[]

In [38]:
for i in range(0, len(text) - sent_length, step_size):
    sent.append(text[i : i + sent_length])
    new_char.append(text[i + sent_length])

In [39]:
new_char

[' ',
 'e',
 'l',
 'r',
 'o',
 'e',
 'r',
 'h',
 ' ',
 ' ',
 'e',
 'h',
 ' ',
 ' ',
 'm',
 'h',
 '\n',
 'l',
 'r',
 'o',
 'e',
 ' ',
 's',
 'v',
 '.',
 'f',
 's',
 'c',
 'i',
 'n',
 'f',
 's',
 ' ',
 'u',
 'n',
 ' ',
 'i',
 ' ',
 'r',
 'u',
 'i',
 'c',
 'e',
 'e',
 'm',
 't',
 't',
 ' ',
 'o',
 'e',
 '\n',
 'l',
 'w',
 'k',
 'w',
 ',',
 'e',
 'n',
 "'",
 '\n',
 'i',
 't',
 'i',
 'z',
 ':',
 'e',
 'u',
 'k',
 'l',
 'i',
 ' ',
 'd',
 'e',
 'l',
 'a',
 ' ',
 'r',
 'a',
 'o',
 ' ',
 'n',
 'r',
 'e',
 'i',
 't',
 ' ',
 'r',
 'c',
 '\n',
 'l',
 '\n',
 ' ',
 'r',
 't',
 'k',
 'g',
 'n',
 ';',
 'e',
 'i',
 'b',
 'd',
 'e',
 'a',
 'y',
 'a',
 'y',
 '\n',
 'c',
 'd',
 'i',
 'z',
 ':',
 'n',
 'w',
 'd',
 'g',
 'd',
 'i',
 'z',
 's',
 '\n',
 'r',
 ' ',
 't',
 'e',
 '\n',
 ' ',
 'e',
 'c',
 'u',
 'e',
 'p',
 'r',
 'i',
 'z',
 's',
 't',
 ' ',
 't',
 'c',
 'n',
 'g',
 'd',
 'w',
 't',
 'u',
 'o',
 't',
 's',
 'f',
 't',
 'o',
 'w',
 'l',
 'r',
 'i',
 'e',
 's',
 'i',
 't',
 'y',
 'o',
 'd',
 'i',
 '

In [40]:
x = np.zeros((len(sent) , sent_length , len(char)) , dtype = bool)
y = np.zeros((len(sent) , len(char)))

In [52]:
for i, satz in enumerate(sent):
    for t, char1 in enumerate(satz):
        x[i, t, char_to_ind[char1]] = 1
    y[i, char_to_ind[new_char[i]]] = 1

In [54]:
model = Sequential()
model.add(LSTM(units = 128, input_shape = (sent_length , len(char))))
model.add(Dense(len(char)))
model.add(Activation('softmax'))

In [55]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop')

In [56]:
model.fit(x, y, epochs=4, batch_size=256)

Epoch 1/4
1453/1453 [==============================] - 273s 187ms/step - loss: 2.4275
Epoch 2/4
1453/1453 [==============================] - 270s 186ms/step - loss: 2.0393
Epoch 3/4
1453/1453 [==============================] - 257s 177ms/step - loss: 1.8881
Epoch 4/4
1453/1453 [==============================] - 260s 179ms/step - loss: 1.7927


In [57]:
model.save('terxtgen.keras')

In [58]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [60]:
def gen_text(length, temprature):
    start = random.randint(0, len(text) - sent_length - 1)
    generated = ''
    sentence = text[start : start + sent_length]
    generated += sentence
    for i in range(length):
        x = np.zeros((1 , sent_length, len(char)))
        for t, charectar in enumerate(sentence):
            x[0 , t , char_to_ind[charectar]] = 1
        
        prediction = model.predict(x , verbose = 0)[0]
        next_ind = sample(prediction, temprature)
        next_char = ind_to_char[next_ind]
        
        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [63]:
print(gen_text(300,0.6))

 first unroof'd the city,
ere so prevail'd with me: it will in time
win upon power and throw forth good.

lecio:
thes wall pervouth he world not the sirved him.

ferrouman:
i have horde propore, a all wo lo do hee.

butinis:
it by the condet of the mare with us manter:
it the shall she the cack not gouddery with thoue senser ald firseed him
he prospore his must afole be not that son the some
villo
